# Introduction

Summary

analysis of Singapore housing price 
predict singapore price moving forward 

analyse the saving amount needed by future generations to afford their own apartment 

resale price = label / target


1. Import Library

In [13]:
# Data Handling
import pandas as pd 
import numpy as np

import re
from scipy.stats import skew

# Data Cleaning & Processing
##from sklearn.model_selection import train_test_split
##from sklearn.model_selection import KFold


# Data Visualisation
##import matplotlib as plt
##? plotly
##?seaborn
##?bokeh


# Model
##from sklearn.ensemble import RandomForestClassifier
##from sklearn.neighbors import KNeighborsClassifier
##svm, logisitic regression, xgbt, et, xgbtlite 

##from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, precision_score, recall_score



2. Load Data

In [3]:
df = pd.read_csv("/Users/zhakaeric-m1air/Library/CloudStorage/GoogleDrive-ericngzh96@gmail.com/My Drive/Private/Education/Data Science Trial & Projects/Github-Repository-Data/#Classification-Regression/##Singapore Housing Price 2017-2022/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
134163,2022-08,YISHUN,5 ROOM,850,YISHUN ST 81,01 TO 03,122.0,Improved,1988,65 years,570000.0
134164,2022-08,YISHUN,EXECUTIVE,325,YISHUN CTRL,07 TO 09,146.0,Maisonette,1988,65 years 04 months,768888.0
134165,2022-08,YISHUN,EXECUTIVE,355,YISHUN RING RD,07 TO 09,150.0,Maisonette,1988,65 years 04 months,778000.0
134166,2022-08,YISHUN,EXECUTIVE,355,YISHUN RING RD,07 TO 09,146.0,Maisonette,1988,65 years 04 months,838000.0


In [4]:
df.describe()

,floor_area_sqm,lease_commence_date,resale_price
count,134168.000000,134168.000000,1.341680e+05
mean,97.765356,1995.325487,4.706689e+05
std,24.098741,13.602076,1.629509e+05
min,31.000000,1966.000000,1.400000e+05
25%,82.000000,1985.000000,3.500000e+05
50%,94.000000,1996.000000,4.400000e+05
75%,113.000000,2006.000000,5.550000e+05
max,249.000000,2019.000000,1.418000e+06


In [5]:
# data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134168 entries, 0 to 134167
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                134168 non-null  object 
 1   town                 134168 non-null  object 
 2   flat_type            134168 non-null  object 
 3   block                134168 non-null  object 
 4   street_name          134168 non-null  object 
 5   storey_range         134168 non-null  object 
 6   floor_area_sqm       134168 non-null  float64
 7   flat_model           134168 non-null  object 
 8   lease_commence_date  134168 non-null  int64  
 9   remaining_lease      134168 non-null  object 
 10  resale_price         134168 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 11.3+ MB


3. Data Preparation
- check missing 
- data cleaning if any + methodology (remove/insert)

In [6]:
# check NULLs 
print(df.isnull().sum())

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64


In [9]:
# Function to convert years and months to years
def convert_to_years(lease):
    # Use regular expressions to extract years and months
    years_match = re.search(r'(\d+) years?', lease)
    months_match = re.search(r'(\d+) months?', lease)
    
    years = int(years_match.group(1)) if years_match else 0
    months = int(months_match.group(1)) if months_match else 0
    
    # Convert months to a fraction of years
    months_in_years = months / 12
    
    return years + months_in_years

# Apply the function to the DataFrame
df['remaining_lease_years'] = df['remaining_lease'].apply(convert_to_years)

# Display the DataFrame
print(df)


          month        town  flat_type block        street_name storey_range  \
0       2017-01  ANG MO KIO     2 ROOM   406  ANG MO KIO AVE 10     10 TO 12   
1       2017-01  ANG MO KIO     3 ROOM   108   ANG MO KIO AVE 4     01 TO 03   
2       2017-01  ANG MO KIO     3 ROOM   602   ANG MO KIO AVE 5     01 TO 03   
3       2017-01  ANG MO KIO     3 ROOM   465  ANG MO KIO AVE 10     04 TO 06   
4       2017-01  ANG MO KIO     3 ROOM   601   ANG MO KIO AVE 5     01 TO 03   
...         ...         ...        ...   ...                ...          ...   
134163  2022-08      YISHUN     5 ROOM   850       YISHUN ST 81     01 TO 03   
134164  2022-08      YISHUN  EXECUTIVE   325        YISHUN CTRL     07 TO 09   
134165  2022-08      YISHUN  EXECUTIVE   355     YISHUN RING RD     07 TO 09   
134166  2022-08      YISHUN  EXECUTIVE   355     YISHUN RING RD     07 TO 09   
134167  2022-08      YISHUN  EXECUTIVE   643       YISHUN ST 61     10 TO 12   

        floor_area_sqm      flat_model 

4. Exploratory Data Analysis 

- descriptive stat
- histo
- correlation 
- identify top factors for fraud 


In [11]:
# Descriptive Statistics 
stat = df.describe()

stat

,floor_area_sqm,lease_commence_date,resale_price,remaining_lease_years
count,134168.000000,134168.000000,1.341680e+05,134168.000000
mean,97.765356,1995.325487,4.706689e+05,74.818882
std,24.098741,13.602076,1.629509e+05,13.526921
min,31.000000,1966.000000,1.400000e+05,43.500000
25%,82.000000,1985.000000,3.500000e+05,64.000000
50%,94.000000,1996.000000,4.400000e+05,74.916667
75%,113.000000,2006.000000,5.550000e+05,85.916667
max,249.000000,2019.000000,1.418000e+06,97.750000


In [15]:


#check skewness 
selected_var = ['floor_area_sqm','lease_commence_date','resale_price','remaining_lease_years']
skewness = df[selected_var].skew()

print("Skewness:", skewness)


#check kurtosis 


Skewness: floor_area_sqm           0.260655
lease_commence_date      0.074543
resale_price             0.998013
remaining_lease_years    0.014429
dtype: float64


In [ ]:
#check qq plot 

#check normal distribution 

In [ ]:
#check correlation 

# correlation heatmap 


In [ ]:
# n-fold cross validation
n = 5
kf = KFold(n_splits=n, shuffle=True, random_state=0)


In [ ]:
#train test split 

#remarks: NA since using K-CV

Models
- create 
- evaluate

regression rf nn 

In [ ]:
# Create Model 1 


In [ ]:
# Evaluate Model 1 

accuracy recall 

In [ ]:
# ROC Graph 

In [ ]:
# Feature Importance 

In [ ]:
# confusion matrix 


Limitations 